In [54]:
import pandas as pd
from pmaw import PushshiftAPI
import re
import numpy as np
api = PushshiftAPI()

import datetime
from datetime import timedelta

In [13]:
def get_field(field, json):
    items = len(json)
    results = [json[i].get(field) for i in range(items)]
    return results

def get_df(json,fields):
    df = pd.DataFrame()
    for field in fields:
        values = get_field(field,json)
        df[field] = values
    return df

In [5]:
subreddit1 = api.search_submissions(subreddit="askwomen", limit=10000)

INFO:pmaw.PushshiftAPIBase:Checkpoint:: Success Rate: 100.00% - Requests: 100 - Batches: 10 - Items Remaining: 84
INFO:pmaw.PushshiftAPIBase:Total:: Success Rate: 100.00% - Requests: 101 - Batches: 11 - Items Remaining: 0


In [20]:
sub1_posts = [post for post in subreddit1]

In [40]:
subreddit2 = api.search_submissions(subreddit="askmen", limit=10000)

INFO:pmaw.PushshiftAPIBase:Checkpoint:: Success Rate: 100.00% - Requests: 100 - Batches: 10 - Items Remaining: 101
INFO:pmaw.PushshiftAPIBase:Total:: Success Rate: 100.00% - Requests: 102 - Batches: 11 - Items Remaining: 0


In [41]:
sub2_posts = [post for post in subreddit2]

In [8]:
fields = ['id','subreddit','created_utc',
          'is_video',
          'spoiler',
          'is_self',
          'score',
          'is_original_content',
          'is_created_from_ads_ui',
          'media_only',
          'over_18',
          'num_comments',
          'num_crossposts',
          'author',
          'author_premium',
          'title',
          'selftext']

In [25]:
df1 = get_df(sub1_posts, fields)
print(df1.shape)
df1.head(3)

(10000, 17)


,id,subreddit,created_utc,is_video,spoiler,is_self,score,is_original_content,is_created_from_ads_ui,media_only,over_18,num_comments,num_crossposts,author,author_premium,title,selftext
0,n4c2kk,AskWomen,1620089308,False,False,True,1,False,None,False,False,5,0,essiemay7777777,False,What is a fragrance you love or you get a lot ...,
1,n4c0tz,AskWomen,1620089165,False,False,True,1,False,None,False,False,9,0,Toiljest,False,Women of reddit (I'm sorry if this has been as...,
2,n4bymt,AskWomen,1620088970,False,False,True,1,False,None,False,False,2,0,lemeurice,False,What is this called? What is going on? Is this...,[removed]


In [42]:
df2 = get_df(sub2_posts, fields)
print(df2.shape)
df2.head(3)

(10000, 17)


,id,subreddit,created_utc,is_video,spoiler,is_self,score,is_original_content,is_created_from_ads_ui,media_only,over_18,num_comments,num_crossposts,author,author_premium,title,selftext
0,mf0rtm,AskMen,1616935459,False,False,True,1,False,None,False,True,2,0,polrotti,False,Why is difficult for me to ejaculate when my w...,[removed]
1,mf0mlx,AskMen,1616934856,False,False,True,1,False,None,False,False,0,0,Missmel194,False,Buying a comedian a drink.,[removed]
2,mf0bwk,AskMen,1616933649,False,False,True,1,False,None,False,False,2,0,ShylockWalker,False,Why are all the topics on this subreddit alway...,[removed]


In [43]:
df = pd.concat([df1,df2])
print(df.shape)
df.sample(3)

(20000, 17)


,id,subreddit,created_utc,is_video,spoiler,is_self,score,is_original_content,is_created_from_ads_ui,media_only,over_18,num_comments,num_crossposts,author,author_premium,title,selftext
9960,o2da5a,AskMen,1623979735,False,False,True,1,False,False,False,False,2,0,VicktorVector,False,How many of us can Identify with this?,[removed]
8343,n0lhvi,AskWomen,1619634800,False,False,True,1,False,None,False,False,2,0,zoeb3456,False,Who has experience this before? manipulative men,[removed]
8998,nag5cl,AskWomen,1620792847,False,False,True,1,False,None,False,False,2,0,AjaxDrowsy,False,"For women who date men, what would you feel ab...",[removed]


In [48]:
df.subreddit.value_counts()

AskWomen    10000
AskMen      10000
Name: subreddit, dtype: int64

In [49]:
df['created_utc'] = [datetime.datetime.fromtimestamp(i) for i in df['created_utc']]

In [50]:
#https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

In [55]:
df = df.fillna('')
df['selftext'] = df['selftext'].replace('[removed]', '')
df['all_text'] = df['title'] + ' ' + df['selftext']

#get string lengths
df['title_length'] = [len(i) for i in df['title']]
df['selftext_length'] = [len(i) for i in df['selftext']]
df['all_text_length'] = [len(i) for i in df['all_text']]

#get word counts
df['title_words'] = [len(i.split()) for i in df['title']]
df['selftext_words'] = [len(i.split()) for i in df['selftext']]
df['all_text_words'] = [len(i.split()) for i in df['all_text']]

#no link
df['no_links_text'] = [remove_urls(i) for i in df['all_text']]

df['contains_link'] = np.where(df['all_text'].str.contains('https'),1,0)

df.sample(3)

,id,subreddit,created_utc,is_video,spoiler,is_self,score,is_original_content,is_created_from_ads_ui,media_only,...,selftext,all_text,title_length,selftext_length,all_text_length,title_words,selftext_words,all_text_words,no_links_text,contains_link
4701,o8wgby,AskMen,2021-06-27 08:47:54,False,False,True,1,False,False,False,...,,Marks on arm after shaving,26,0,27,5,0,5,Marks on arm after shaving,0
5731,pq1dw9,AskWomen,2021-09-17 10:37:11,False,False,True,1,False,False,False,...,,"What’s something you got into (hobby, interest...",146,0,147,27,0,27,"What’s something you got into (hobby, interest...",0
3031,ml4aoa,AskWomen,2021-04-06 01:17:58,False,False,True,1,False,,False,...,"I'm 24 now, but ever since I was in middle sch...","Why do women assume I'm gay? I'm 24 now, but e...",28,105,134,6,21,27,"Why do women assume I'm gay? I'm 24 now, but e...",0


In [56]:
df.to_csv("../data/women_men.csv")

In [57]:
t = 'something here?'
t.replace("?","")

'something here'

In [59]:
df.shape[0]

20000